# Document translation batch (asynchronous mode)

https://learn.microsoft.com/en-us/azure/ai-services/translator/document-translation/how-to-guides/use-rest-api-programmatically?tabs=csharp

<img src="https://github.com/retkowsky/azure-ai-translator/blob/main/logo.jpg?raw=true" width=200>

In [1]:
import datetime
import json
import os
import requests
import sys
import time

from azure.ai.translation.document import DocumentTranslationClient
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 05-Mar-2025 11:17:25


## Azure AI Translator credentials

In [4]:
load_dotenv("azure.env")

azure_ai_translator_key = os.getenv("AZURE_AI_TRANSLATION_KEY")
azure_ai_translator_endpoint = os.getenv("AZURE_AI_TRANSLATION_ENDPOINTDOCUMENT")
azure_ai_translator_region = os.getenv("AZURE_AI_TRANSLATION_REGION")

blob_results_connection_string = os.getenv("connection_string")

## Shared Access Signatures (SAS)

In order to read and write from your blobs in your Azure storage account you need to do these steps:<br><br>
1 Go the container from your storage account<br>
2 Go to **"Settings / Shared Access Tokens"**<br>
3 Check the **"permissions"**<br>
4 Check the **"expiry"** informations<br>
5 Then click on **"Generate SAS Token and URL".**<br>
6 You will have your **"Blob SAS token"** and the **"Blob SAS URL"**.<br>

- Below we have two blob storage (one for the source documents, another one for the translated documents).
- For the source blob storage you should define the **"Read"** and **"List"** permissions (step 3).
- For the results blob strage you should define the **"Write"** and **"List"** permissions (step 3).

<img src="https://github.com/retkowsky/azure-ai-translator/blob/main/SAS.jpg?raw=true">

In [5]:
# Source blob SAS
sourceblobsasurl = os.getenv("sourceblobsasurl")
sourceblobsastoken = os.getenv("sourceblobsastoken")
targetblobsasurl = os.getenv("targetblobsasurl")
targetblobsastoken = os.getenv("targetblobsastoken")

sourceUri = sourceblobsasurl + sourceblobsastoken
targetUri = targetblobsasurl + targetblobsastoken

## Azure AI Translator client

In [6]:
# initialize a new instance of the DocumentTranslationClient object to interact with the Document Translation feature
translator_client = DocumentTranslationClient(azure_ai_translator_endpoint, AzureKeyCredential(azure_ai_translator_key))

## Example

In [7]:
targetLanguage = "fr"

> https://aka.ms/TranslatorLanguageCodes 

In [8]:
# include source and target locations and target language code for the begin translation operation
print(
    f"Starting to translate the documents to language:{targetLanguage} ...")

start = time.time()

poller = translator_client.begin_translation(sourceUri, targetUri, targetLanguage)
result = poller.result()

elapsed = time.time() - start
print("\nElapsed time: " + time.strftime("%H:%M:%S.{}".format(str(elapsed %
      1)[2:])[:15], time.gmtime(elapsed)))

Starting to translate the documents to language:fr ...

Elapsed time: 00:01:00.442654


## Results

In [9]:
print("\033[1;31;34m")
print("Status: {}".format(poller.status()))
print("Created on: {}".format(poller.details.created_on))
print("Last updated on: {}".format(poller.details.last_updated_on))
print(
    "\nTotal number of translations on documents: {}".format(
        poller.details.documents_total_count
    )
)

print("  {} failed".format(poller.details.documents_failed_count))
print("  {} succeeded".format(poller.details.documents_succeeded_count))


Status: Succeeded
Created on: 2025-03-05 11:17:26.188541+00:00
Last updated on: 2025-03-05 11:17:49.795860+00:00

Total number of translations on documents: 4
  0 failed
  4 succeeded


In [10]:
nb = 1

for document in result:
    print("- Document", nb)
    print("Document ID: {}".format(document.id))
    print("Document status: {}".format(document.status))

    if document.status == "Succeeded":
        print("Source document location: {}".format(
            document.source_document_url))
        print(
            "Translated document location: {}".format(
                document.translated_document_url)
        )
        print("Translated to language: {}\n".format(document.translated_to))
        nb += 1

- Document 1
Document ID: 009bb304-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/termsconditions_en.pdf
Translated document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs-translated/termsconditions_en.pdf
Translated to language: fr

- Document 2
Document ID: 009bb303-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/releasenotes_en.pdf
Translated document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs-translated/releasenotes_en.pdf
Translated to language: fr

- Document 3
Document ID: 009bb302-0000-0000-0000-000000000000
Document status: Succeeded
Source document location: https://azurestorageaccountsr.blob.core.windows.net:443/docs/Microsoft%20Translator%20Customer%20Ready%20Deck%20v4.1.pptx
Translated document location: https://azurestorageaccountsr.bl

## Translated files

In [11]:
container_name = "docs-translated"

blob_service_client = BlobServiceClient.from_connection_string(blob_results_connection_string)
container_client = blob_service_client.get_container_client(container_name)
fileslist = container_client.list_blobs()

nb = 0
print("\033[1;31;34m")

for file in fileslist:
    print(f"{nb+1} Document: {file.name}")
    print(f"Date: {file.last_modified.strftime('%Y-%m-%d %H:%M:%S')}")
    size_mb = file.size / (1024 * 1024)
    print(f"Size: {file.size} bytes ({size_mb:.2f} MB)\n")
    nb += 1


1 Document: AutoGen A Multi-Agent Framework for Enabling Next-Gen AI Applications.msg
Date: 2025-03-05 11:17:49
Size: 128512 bytes (0.12 MB)

2 Document: Microsoft Translator Customer Ready Deck v4.1.pptx
Date: 2025-03-05 11:17:57
Size: 16973181 bytes (16.19 MB)

3 Document: releasenotes_en.pdf
Date: 2025-03-05 11:17:55
Size: 257886 bytes (0.25 MB)

4 Document: termsconditions_en.pdf
Date: 2025-03-05 11:17:57
Size: 387251 bytes (0.37 MB)



In [12]:
print(f"Total number of translated files = {nb}")

Total number of translated files = 4
